[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/antoniotrapote/chord-prediction-tfm/blob/main/anexos/notebooks/01_análisis_exploratorio/01_dataset_compile.ipynb)
[![View on GitHub](https://img.shields.io/badge/View_on-GitHub-black?logo=github)](https://github.com/antoniotrapote/chord-prediction-tfm/blob/main/anexos/notebooks/01_análisis_exploratorio/01_dataset_compile.ipynb)

In [1]:
import os
import pandas as pd

## Función para extraer los datos de un archivo .txt de canción

In [2]:
import urllib.request

def parse_song_file(filepath):
    """Esta función parsea los archivos de canción en formato .txt"""
    if filepath.startswith('http'):
        # Es una URL
        with urllib.request.urlopen(filepath) as response:
            content = response.read().decode('utf-8')
            lines = content.splitlines()
    else:
        # Es un archivo local
        with open(filepath, 'r', encoding='utf-8') as f:
            lines = f.readlines()
    
    data = {}
    chord_lines = []
    for line in lines:
        line = line.strip()
        if line.startswith('Title ='):
            data['title'] = line.split('=',1)[1].strip()
        elif line.startswith('ComposedBy ='):
            data['composedby'] = line.split('=',1)[1].strip()
        elif line.startswith('DBKeySig ='):
            data['key'] = line.split('=',1)[1].strip()
        elif line.startswith('TimeSig ='):
            data['timesig'] = line.split('=',1)[1].strip()
        elif line.startswith('Bars ='):
            bars = line.split('=',1)[1].strip()
            data['bars'] = int(bars) if bars.isdigit() else bars
        elif line:
            chord_lines.append(line)
    data['chordprog'] = '\n'.join(chord_lines)
    return data

In [3]:
USER = "antoniotrapote"
REPO = "chord-prediction-tfm"
BRANCH = "main"
PATH_IN_REPO = "anexos/data/SongDB/Songs%5B%23%2CA-G%5D/26-2.txt"
URL = f"https://raw.githubusercontent.com/{USER}/{REPO}/{BRANCH}/{PATH_IN_REPO}"

# Prueba con una sola canción antes de procesar todo el dataset
test_file = URL
test_record = parse_song_file(test_file)
for k, v in test_record.items():
    print(f'{k}:\n{v}')

title:
26-2
composedby:
John Coltrane
key:
F
timesig:
4 4
bars:
32
chordprog:
FM7 Ab7 | DbM7 E7 | AM7 C7 | Cm7 F7 |
BbM7 C#7 | F#M7 A7 | Dm7 G7 | Gm7 C7 |
FM7 Ab7 | DbM7 E7 | AM7 C7 | Cm7 F7 |
BbM7 Ab7 | DbM7 E7/B | AM7 C7 | FM7 |
Cm7 F7 | Em7 A7 | DM7 F7 | BbM7 |
Ebm7 | Ab7 | DbM7 | Gm7 C7 |
FM7 Ab7 | DbM7 E7 | AM7 C7 | Cm7 F7 |
BbM7 Ab7 | DbM7 E7/B | AM7 C7 | FM7 |


## Recorremos todos los archivos .txt y compilamos el DataFrame

**Nota: La siguiente celda recorre todos los archivos .txt alojados en el directorio `../../data/SongDB` del repositorio. Si desea ejecutarla, se recomienda clonar el repositorio y ejecutar este notebook en local para evitar problemas con el acceso a los archivos.**

La intención no es tanto repetir el proceso sino sencillamente dejar constancia del método que se siguió originalmente.

In [10]:
song_dir = '../../data/SongDB'
song_files = []
for root, dirs, files in os.walk(song_dir):
    for file in files:
        if file.endswith('.txt'):
            song_files.append(os.path.join(root, file))

records = []
for f in song_files:
    try:
        record = parse_song_file(f)
        records.append(record)
    except Exception as e:
        print(f'Error en {f}: {e}')

df = pd.DataFrame(records)
df.head()

,title,composedby,key,timesig,bars,chordprog
0,Lullaby of Birdland,George Shearing,Ab,4 4,32,Fm7 Dm7b5 | G7b9 C7b9 | Fm7 DbM7 | Bbm7 Eb7 |\...
1,It's A Most Unusual Day,Jimmy McHugh and HYarold Adamson,G,3 4,72,F#/G F#/G G | Em7 | Am7 | D7 |\nF#/G F#/G G | ...
2,Jump Monk,Charles Mingus,Ab,4 4,54,Fm7 DbM7 | G7b5 C7 | Fm7 DbM7 | G7b5 C7 |\nFm7...
3,Nuages,Django Reinhardt and Jacques Larme,G,4 4,32,Bbm7 Eb7 | Am7b5 D7b9 | G6 Am7 | Bm7 |\nBbm7 E...
4,Love Me Do,John Lennon and Paul McCartney,G,4 4,48,G | C | G | C |\nG | C | C | C |\nC | G | C | ...


In [11]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2614 entries, 0 to 2613
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       2613 non-null   object
 1   composedby  2614 non-null   object
 2   key         2613 non-null   object
 3   timesig     2614 non-null   object
 4   bars        2614 non-null   int64 
 5   chordprog   2614 non-null   object
dtypes: int64(1), object(5)
memory usage: 122.7+ KB
None


## Breve exploración del dataset obtenido

### Revisamos los valores nulos
Corregimos manualmente un par de lineas, con erratas leves.

In [12]:
# mostrar filas con Nan
df[df.isna().any(axis=1)]

,title,composedby,key,timesig,bars,chordprog
390,On A Misty Night,Tadd Dameron,NaN,4 4,32,DB Key = Eb\nEbM7 Gm7 | Cm7 Gm7 F#m7 Fm7 | Fm7...
2171,NaN,Dizzy Gillespie,C,4 4,32,mpTitle = Bebop\nFm7 Go7 | Fm7/Ab Go7 | Fm7/C ...


In [13]:
df[df['title']=='On A Misty Night']

,title,composedby,key,timesig,bars,chordprog
390,On A Misty Night,Tadd Dameron,NaN,4 4,32,DB Key = Eb\nEbM7 Gm7 | Cm7 Gm7 F#m7 Fm7 | Fm7...


In [14]:
# eliminamos DB Key = Eb\n de la columna 'chordprog' en el registro 390
df.at[390, 'chordprog'] = df.at[390, 'chordprog'].replace('DB Key = Eb\n', '')
# Asignamos Eb a la columna 'key' en el registro 390
df.at[390, 'key'] = 'Eb'
df[df['title']=='On A Misty Night']

,title,composedby,key,timesig,bars,chordprog
390,On A Misty Night,Tadd Dameron,Eb,4 4,32,EbM7 Gm7 | Cm7 Gm7 F#m7 Fm7 | Fm7 | Fm7 |\nDb7...


In [15]:
# eliminamos mpTitle = Bebop\n de la columna 'chordprog' en el registro 2171
df.at[2171, 'chordprog'] = df.at[2171, 'chordprog'].replace('mpTitle = Bebop\n', '')
# Asignamos Bebop a la columna 'title' en el registro 2171
df.at[2171, 'title'] = 'Bebop'
df[df['title']=='Bebop']

,title,composedby,key,timesig,bars,chordprog
2171,Bebop,Dizzy Gillespie,C,4 4,32,Fm7 Go7 | Fm7/Ab Go7 | Fm7/C | Gm7b5 C7b9 |\nF...


## Echamos un vistazo a la columna 'key'
Observamos que todas los registros indican una tonalidad mayor. Esto es una convención. Pero trataremos de mejorar esta representación incluyendo también tonalidades menores donde sea apropiado.

In [16]:
df['key'].value_counts()

key
C     857
F     488
Eb    401
Bb    349
G     214
Ab    144
Db     65
D      47
A      19
Gb     14
E      13
B       2
U       1
Name: count, dtype: int64

In [17]:
# Mostrar canciones con key == U
rarekey = df[df['key'] == 'U']['chordprog']
for line in rarekey:
    print(line)

Cm7b9 | Dm7b9 | Ebm7b9 | Em7b9 |
Cm7b9 | Dm7b9 | Ebm7b9 | Em7b9 |
Cm7b9 | Dm7b9 | Ebm7b9 | Em7b9 |
Cm7b9 | Dm7b9 | Ebm7b9 | F9sus4 |


Se trata de una pieza atonal (modal), por eso es normal que no tenga asignada una tonalidad. Vamos a prescindir de este registro para nuestro dataset.

In [18]:
# Eliminamos el registro donde key == U
df = df[df['key'] != 'U']

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2613 entries, 0 to 2613
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       2613 non-null   object
 1   composedby  2613 non-null   object
 2   key         2613 non-null   object
 3   timesig     2613 non-null   object
 4   bars        2613 non-null   int64 
 5   chordprog   2613 non-null   object
dtypes: int64(1), object(5)
memory usage: 142.9+ KB


## Exportamos el dataset de canciones compiladas

Si ejecuta la siguiente celda, se sobreescribirá el archivo existente `songdb.csv` en el directorio `../../data/`.

In [ ]:
# df.to_csv('../../data/songdb.csv', index=False)